# Inżynieria lingwistyczna
Ten notebook jest oceniany półautomatycznie. Nie twórz ani nie usuwaj komórek - struktura notebooka musi zostać zachowana. Odpowiedź wypełnij tam gdzie jest na to wskazane miejsce - odpowiedzi w innych miejscach nie będą sprawdzane (nie są widoczne dla sprawdzającego w systemie).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

---

# Moduł 5: Statystyczne tłumaczenie maszynowe

## Zadanie 1
Zadanie polega na zaimplementowaniu algorytmu Expectation-Maximization w modelu IBM Model 1 do przypasowywania słów. Będzie to fragment modelu, który tłumaczyć będzie z hiszpańskiego na angielski. 

UWAGA: Specjalny token "NULL" pomijamy w implementacji.

Dany jest mini-korpus równoległy angielsko-hiszpański
- "green house" "casa verde"
- "the house" "la casa"
- "the green house" "la casa verde"


In [ ]:
import itertools
english = [["green","house"], ["the","house"], ["the", "green", "house"]]
spanish = [["casa", "verde"], ["la", "casa"], ["la", "casa", "verde"]]

W dalszych funkcjach przydatne może być wyznaczenie słownika czyli zbioru słów z korpusu dla danego języka.

In [ ]:
def get_vocabulary(corpus):
    """
    Funkcja zwracająca listę unikalnych słów z korpusu podanego w formacie zmiennej english i spanish
    """
    return list(set(x for l in corpus for x in l))

In [ ]:
from nose.tools import assert_set_equal
assert_set_equal(set(get_vocabulary(english)), set(["the", "green", "house"]))

Zainicjalizuj rozkład prawdopodobieństwa tłumaczenia słów rozkładem jednorodnym. Ponieważ zależy nam na prostocie implementacji (a nie efektywności) możemy to prawdopodobieństwo zaimplementować jako zwykły słownik, który będzie przyjmował na wejściu krotkę dwóch słów. Słownik nazwij `translation_prob` z kluczami (słowo_es, słowo_en).

In [ ]:
def initalize_translation_prob(corpus1, corpus2):
    translation_prob = {}

    corpus1_words_list = get_vocabulary(corpus1)
    corpus2_words_list = get_vocabulary(corpus2)

    prob_value = 1 / len(corpus1_words_list)

    for word_corpus2 in corpus2_words_list:
        for word_corpus1 in corpus1_words_list:
            translation_prob[(word_corpus2, word_corpus1)] = prob_value

    return translation_prob
translation_prob = initalize_translation_prob(english, spanish)

Wypisz zaincjalizowany słownik, żeby upewnić się że wynik jest prawidłowy.

In [ ]:
translation_prob

Zaimplementuj pierwszy krok algorytmu EM. Wyznacz wartości oczekiwane zmiennych przypisania słowa we wszystkich zdaniach w korpusie (oznaczane na wykładzie jako `a`).

In [ ]:
from collections import defaultdict

def calculate_expectation(corpora1, corpora2, translation_prob):
    """
    Procedura wykonująca krok "E" algorytmu EM
    Wynikiem powinny być wartości oczekiwane dla zmiennej przypisań słów w zdaniach 
    (reprezentacja dowolna, nie weryfikowana przez sprawdzarkę)
    """
    expected_values = {}
    
    for k, (sent_corp_1, sent_corp_2) in enumerate(zip(corpora1, corpora2)):
        den = defaultdict(lambda: 0.0)
        
        for word_corp_1 in sent_corp_1:
            for word_corp_2 in sent_corp_2:
                actual_prob = translation_prob[(word_corp_2, word_corp_1)]
                den[word_corp_1] += actual_prob
                expected_values[(k, word_corp_2, word_corp_1)] = actual_prob
        
        for word_corp_1 in sent_corp_1:
            for word_corp_2 in sent_corp_2:
                expected_values[(k, word_corp_2, word_corp_1)] /= den[word_corp_1]
                
    return expected_values
        
assigment_expected_values = calculate_expectation(english, spanish, translation_prob)

Wypisz wartości oczekiwane zmiennych przypisań, aby zobaczyć jak wyglądają. Powinny one również prezentować całkowity brak wiedzy o przypisaniach (rozkłady jednorodne).

In [ ]:
assigment_expected_values

Zaimplementuj drugi krok algorytmu EM. Wyznacz nowe `translation_prob` na podstawie oczekiwanych wartości zmiennych przypisań.

In [ ]:
from collections import defaultdict

def calculate_maximization(corpora1, corpora2, assigment_expected_values):
    translation_prob = defaultdict(lambda: 0.0)
    word_prob = defaultdict(lambda: 0.0)
    
    for (k, word_corp_2, word_corp_1), prob in assigment_expected_values.items():
        translation_prob[(word_corp_2, word_corp_1)] += prob
        word_prob[word_corp_1] += prob
        
    for (word_corp_2, word_corp_1), prob in translation_prob.items():
        translation_prob[(word_corp_2, word_corp_1)] = prob / word_prob[word_corp_1] 
    return translation_prob

translation_prob = calculate_maximization(english, spanish, assigment_expected_values)

In [ ]:
from nose.tools import assert_almost_equal
assert_almost_equal(translation_prob[('casa', 'house')], 4/9.)
assert_almost_equal(translation_prob[('la', 'house')], 5/18.)

Wywołaj w pętli 10 kroków algorytmu EM i zaobserwuj jak zmieniają się prawdopodobieństwa dla tłumacznienia "house".

In [ ]:
for i in range(10):
    assigment_expected_values = calculate_expectation(english, spanish, translation_prob)
    translation_prob = calculate_maximization(english, spanish, assigment_expected_values)
    print([(i,j) for i,j in translation_prob.items() if i[1] == "house"])
    print("---")


Wywołaj algorytm EM na poniższym korpusie.

In [ ]:
english2 = [["the","dog"], ["the","house"], ["the", "green", "house"]]
polish = [["pies"], ["dom"], ["zielony", "dom"]]

In [ ]:
translation_prob = initalize_translation_prob(english2, polish)
# YOUR CODE HERE
for i in range(10):
    assigment_expected_values = calculate_expectation(english2, polish, translation_prob)
    translation_prob = calculate_maximization(english2, polish, assigment_expected_values)
    print([(i,j) for i,j in translation_prob.items() if i[1] == "house"])
    print("---")

Sprawdź jak wyglądają prawdopodobieństwa tłumaczeń po 10 iteracjach.

In [ ]:
translation_prob

Sprawdź czy gdybyś dodał słówko `NULL` to algorytm nauczyłby się wiązać słówko `NULL` na `the`, które nie występuje w języku polskim?

In [ ]:
# YOUR CODE HERE
english2 = [["the","dog"], ["the","house"], ["the", "green", "house"]]
polish_null = [["NULL", "pies"], ["NULL", "dom"], ["NULL", "zielony", "dom"]]

translation_prob = initalize_translation_prob(english2, polish_null)
# YOUR CODE HERE
for i in range(10):
    assigment_expected_values = calculate_expectation(english2, polish_null, translation_prob)
    translation_prob = calculate_maximization(english2, polish_null, assigment_expected_values)
    print([(i,j) for i,j in translation_prob.items() if i[1] == "the"])
    print("---")

In [ ]:
translation_prob

Jeśli wywołałbyś EM dla pierwszego korpusu równoległego (zmienne `english` i `spanish`) i dołączył tokeny `NULL` to EM tłumaczy NULL jako "casa" i "house" jako "casa" z takimi samymi prawdopodobieństwami. Dlaczego?

Być może dlatego że model nie ma świadomości jak używać token NULL i traktuje je jak każde inne słowo i próbuje przypisać mu normalne prawdopodobieństwo na podstawie tego jak często występował w korpusie